# Semantic Segmentation Demo

This is a notebook for running the benchmark semantic segmentation network from the the [ADE20K MIT Scene Parsing Benchchmark](http://sceneparsing.csail.mit.edu/).

The code for this notebook is available here
https://github.com/CSAILVision/semantic-segmentation-pytorch/tree/master/notebooks

It can be run on Colab at this URL https://colab.research.google.com/github/CSAILVision/semantic-segmentation-pytorch/blob/master/notebooks/DemoSegmenter.ipynb

### Environment Setup

First, download the code and pretrained models if we are on colab.

In [2]:
import numpy as np
import pandas as pd
import msgpack
from io import BytesIO

,Class Index,Class Name,% of Image Pixels in


In [4]:
%%bash
# Colab-specific setup
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit 
pip install yacs 2>&1 >> install.log
git init 2>&1 >> install.log
git remote add origin https://github.com/CSAILVision/semantic-segmentation-pytorch.git 2>> install.log
git pull origin master 2>&1 >> install.log
DOWNLOAD_ONLY=1 ./demo_test.sh 2>> install.log

From https://github.com/CSAILVision/semantic-segmentation-pytorch
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports and utility functions

We need pytorch, numpy, and the code for the segmentation model.  And some utilities for visualizing the data.

In [3]:
from google.colab import drive

!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser 
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "My Drive"
!google-drive-ocamlfuse "/content/gdrive/My Drive"

/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force&state=eCKUjze-fGj9kGFy3OFFb0eyHAvDEpuViFduwADg3Mc'
/bin/sh: 1: firefox: not found
/bin/sh: 1: google-chrome: not found
/bin/sh: 1: chromium-browser: not found
/bin/sh: 1: open: not found
Cannot retrieve auth tokens.
Failure("Error opening URL:https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.go

In [5]:
%cd /content/

/content


In [ ]:
"""
#!/bin/bash

# Image and model names
TEST_IMG=ADE_val_00001519.jpg
MODEL_NAME=ade20k-hrnetv2-c1
MODEL_PATH=ckpt/$MODEL_NAME
RESULT_PATH=./

ENCODER=$MODEL_NAME/encoder_epoch_30.pth
DECODER=$MODEL_NAME/decoder_epoch_30.pth

# Download model weights and image
if [ ! -e $MODEL_PATH ]; then
  mkdir -p $MODEL_PATH
fi
if [ ! -e $ENCODER ]; then
  wget -P $MODEL_PATH http://sceneparsing.csail.mit.edu/model/pytorch/$ENCODER
fi
if [ ! -e $DECODER ]; then
  wget -P $MODEL_PATH http://sceneparsing.csail.mit.edu/model/pytorch/$DECODER
fi
if [ ! -e $TEST_IMG ]; then
  wget -P $RESULT_PATH http://sceneparsing.csail.mit.edu/data/ADEChallengeData2016/images/validation/$TEST_IMG
fi

if [ -z "$DOWNLOAD_ONLY" ]
then

# Inference
python3 -u test.py \
  --imgs $TEST_IMG \
  --cfg config/ade20k-hrnetv2.yaml \
  DIR $MODEL_PATH \
  TEST.result ./ \
  TEST.checkpoint epoch_30.pth

fi
"""

In [6]:
! ./demo_test.sh

#! ./content/demo_test.sh


--2022-12-04 07:13:49--  http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-hrnetv2-c1/encoder_epoch_30.pth
Resolving sceneparsing.csail.mit.edu (sceneparsing.csail.mit.edu)... 128.30.195.26
Connecting to sceneparsing.csail.mit.edu (sceneparsing.csail.mit.edu)|128.30.195.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 262518297 (250M)
Saving to: ‘ckpt/ade20k-hrnetv2-c1/encoder_epoch_30.pth’

encoder_epoch_30.pt 100%[===================>] 250.36M  36.1MB/s    in 7.6s    

2022-12-04 07:13:57 (32.9 MB/s) - ‘ckpt/ade20k-hrnetv2-c1/encoder_epoch_30.pth’ saved [262518297/262518297]

--2022-12-04 07:13:57--  http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-hrnetv2-c1/decoder_epoch_30.pth
Resolving sceneparsing.csail.mit.edu (sceneparsing.csail.mit.edu)... 128.30.195.26
Connecting to sceneparsing.csail.mit.edu (sceneparsing.csail.mit.edu)|128.30.195.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4780389 (4.6M)
Saving to: ‘ckpt

In [8]:
# System libs
import os, csv, torch, numpy, scipy.io, PIL.Image, torchvision.transforms
# Our libs
from mit_semseg.models import ModelBuilder, SegmentationModule
from mit_semseg.utils import colorEncode

colors = scipy.io.loadmat('/content/data/color150.mat')['colors']
names = {}
with open('/content/data/object150_info.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        names[int(row[0])] = row[5].split(";")[0]

def visualize_result(img, pred, imagename,index=None,):
    df=pd.DataFrame(columns = ['Class Index', 'Class Name', '% of Image Pixels in'])
    # filter prediction class if requested
    pixs = pred.size
    uniques, counts = np.unique(pred, return_counts=True)
    for idx in np.argsort(counts)[::-1]:
        name = names[uniques[idx] + 1]
        ratio = counts[idx] / pixs * 100
        if ratio !=0:
          df=df.append({'Class Index': uniques[idx]+1,'Class Name':name,'% of Image Pixels in':ratio},ignore_index=True)
          #print("{}  {}: {:.2f}%".format(uniques[idx] + 1,name, ratio))

    #if index is not None:
    #    
    #    pred = pred.copy()
    #    cnt=0
    #
    #    for x in pred.flatten():
    #      if x==index:
    #        cnt+=1

        
        #pred[pred != index] = -1
        
    #percentage=100*cnt/len(pred.flatten())
    #print("percent:",percentage)
    #print(index+1,f'{names[index+1]}:')
        
    # colorize prediction
    pred_color = colorEncode(pred, colors).astype(numpy.uint8)

    # aggregate images and save
    im_vis = numpy.concatenate((img, pred_color), axis=1)
    
    png = Image.fromarray(pred_color)
    #png.putpalette(palette)
    
    png.save('/content/drive/MyDrive/Transformer_Based_Geo-localization/resources/shard_seg_train/' + imagename.replace('.jpg','.png'))
    df.to_csv("/content/drive/MyDrive/Transformer_Based_Geo-localization/resources/shard_seg_train/"+imagename.replace('.jpg','.csv'))
   

    #display(PIL.Image.fromarray(im_vis))

In [ ]:
pred_color=colorEncode(pred, colors).astype(numpy.uint8)

In [ ]:
semanticmat=io.loadmat("/content/gdrive/My Drive/Transformer_Based_Geo-localization/resources/semantic_cagtegories.mat")
semanticmat


In [ ]:
from scipy import io
colors =io.loadmat('data/color150.mat')['colors']
#mat_file = io.loadmat('/home/ukjung18/Transformer_Based_Geo-localization/resources/semantic_cagtegories.mat')
colors.flatten()

In [ ]:
data=pd.read_csv('data/object150_info.csv')
data

In [ ]:
names

## Loading the segmentation model

Here we load a pretrained segmentation model.  Like any pytorch model, we can call it like a function, or examine the parameters in all the layers.

After loading, we put it on the GPU.  And since we are doing inference, not training, we put the model in eval mode.

In [9]:
# Network Builders
net_encoder = ModelBuilder.build_encoder(
    arch='hrnetv2',
    fc_dim=720,
    weights='ckpt/ade20k-hrnetv2-c1/encoder_epoch_30.pth')
net_decoder = ModelBuilder.build_decoder(
    arch='c1',
    fc_dim=720,
    num_class=150,
    weights='ckpt/ade20k-hrnetv2-c1/decoder_epoch_30.pth',
    use_softmax=True)

crit = torch.nn.NLLLoss(ignore_index=-1)
segmentation_module = SegmentationModule(net_encoder, net_decoder, crit)
segmentation_module.eval()
segmentation_module.cuda()

Loading weights for net_encoder
Loading weights for net_decoder


SegmentationModule(
  (encoder): HRNetV2(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn2): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): Synchron

## Load test data

Now we load and normalize a single test image.  Here we use the commonplace convention of normalizing the image to a scale for which the RGB values of a large photo dataset would have zero mean and unit standard deviation.  (These numbers come from the imagenet dataset.)  With this normalization, the limiiting ranges of RGB values are within about (-2.2 to +2.7).

In [9]:
# Load and normalize one image as a singleton tensor batch
pil_to_tensor = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406], # These are RGB mean+std values
        std=[0.229, 0.224, 0.225])  # across a large photo dataset.
])


In [ ]:
import numpy as np
import numpy as np

import matplotlib.pyplot as plt

read = np.asarray(Image.open('/content/image.png'))
print(read)

## Run the Model

Finally we just pass the test image to the segmentation model.

The segmentation model is coded as a function that takes a dictionary as input, because it wants to know both the input batch image data as well as the desired output segmentation resolution.  We ask for full resolution output.

Then we use the previously-defined visualize_result function to render the segmentation map.

In [10]:
def get_image(record):
        return Image.open(BytesIO(record["image"]))



dataset_dir = "/content/gdrive/My Drive/Transformer_Based_Geo-localization/resources/shard_images"
shard_fnames = [dataset_dir + "/"+ x for x in os.listdir(dataset_dir)]
#shard_fname="/content/gdrive/My Drive/Transformer_Based_Geo-localization/resources/shard_seperated/shard_0.msg"
"""
for shard_fname in shard_fnames[19:]:
  with open(shard_fname, "rb") as infile:
          for record in msgpack.Unpacker(infile, raw=False):
              #print(f'Image_id={record["id"]}, lat={record["latitude"]}, lon={record["longitude"]}')
              #print("record['id']:",record["id"])
              #image_id.append(record["id"])
              #lati.append(record["latitude"])
              #longti.append(record["longitude"])
              image = get_image(record)   #----> Image.open(BytesIO(record["image"]))
              """


'\nfor shard_fname in shard_fnames[19:]:\n  with open(shard_fname, "rb") as infile:\n          for record in msgpack.Unpacker(infile, raw=False):\n              #print(f\'Image_id={record["id"]}, lat={record["latitude"]}, lon={record["longitude"]}\')\n              #print("record[\'id\']:",record["id"])\n              #image_id.append(record["id"])\n              #lati.append(record["latitude"])\n              #longti.append(record["longitude"])\n              image = get_image(record)   #----> Image.open(BytesIO(record["image"]))\n              '

In [ ]:
#dataset_dir = "/content/gdrive/My Drive/Transformer_Based_Geo-localization/resources/shard_images"
#shard_fnames = [x for x in os.listdir(dataset_dir)]
shard_fnames

In [ ]:
shard_fnames[525]

'65*38*235576661.jpg'

In [ ]:
shard_fname="/content/gdrive/My Drive/Transformer_Based_Geo-localization/resources/data/enowara/mp16/shards/shard_0.msg"
with open(shard_fname, "rb") as infile:
          for record in msgpack.Unpacker(infile, raw=False):
              #print(f'Image_id={record["id"]}, lat={record["latitude"]}, lon={record["longitude"]}')
              #print("record['id']:",record["id"])
              #image_id.append(record["id"])
              #lati.append(record["latitude"])
              #longti.append(record["longitude"])
              #image = get_image(record)   #----> Image.open(BytesIO(record["image"]))
              #title=record["id"].replace("/",'*')
              #image.save("/content/gdrive/My Drive/Transformer_Based_Geo-localization/resources/shard_images/"+title)
              

KeyboardInterrupt: ignored

In [ ]:
dataset_dir = "/content/gdrive/My Drive/Transformer_Based_Geo-localization/resources/shard_segment2"

In [10]:
import msgpack
from PIL import Image
from io import BytesIO
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load and normalize one image as a singleton tensor batch
pil_to_tensor = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406], # These are RGB mean+std values
        std=[0.229, 0.224, 0.225])  # across a large photo dataset.
])


def get_image(record):
        return Image.open(BytesIO(record["image"]))

# shard_segment

cnt=0
shard_fname="/content/drive/MyDrive/Transformer_Based_Geo-localization/resources/shard_train/shard_0.msg"
with open(shard_fname, "rb") as infile:


  for record in msgpack.Unpacker(infile, raw=False):
    
    #print(f'Image_id={record["id"]}, lat={record["latitude"]}, lon={record["longitude"]}')
    #print("record['id']:",record["id"])
    #image_id.append(record["id"])
    #lati.append(record["latitude"])
    #longti.append(record["longitude"])
      cnt+=1
      image = get_image(record)   #----> Image.open(BytesIO(record["image"]))
      imgs = record["id"].replace('/', '_')

#             pil_image = PIL.Image.open(image).convert('RGB')
      img_original = numpy.array(image)
      img_data = pil_to_tensor(image)
      singleton_batch = {'img_data': img_data[None].cuda()}
      output_size = img_data.shape[1:]
          
            
            
          #pil_image

          # Run the segmentation at the highest resolution.
      with torch.no_grad():
          scores = segmentation_module(singleton_batch, segSize=output_size)

      # Get the predicted scores for each pixel
      _, pred = torch.max(scores, dim=1)
      pred = pred.cpu()[0].numpy()


      visualize_result(img_original, pred, imgs)
   
print(cnt)            

29999


In [ ]:
shard_fnames[0]

'eb*61*7325810506.jpg'

In [ ]:
dir="/content/gdrive/My Drive/Transformer_Based_Geo-localization/resources/shard_images/"
for x in shard_fnames:
  print(dir+x)
  break

/content/gdrive/My Drive/Transformer_Based_Geo-localization/resources/shard_images/e0*88*5003029777.jpg


In [ ]:
dataset_dir = "/content/gdrive/My Drive/Transformer_Based_Geo-localization/resources/shard_segment2"
cnt=0
for x in os.listdir(dataset_dir):
  cnt+=1
print(cnt)

1044


In [ ]:
shard_fnames="/content/gdrive/My Drive/Transformer_Based_Geo-localization/resources/shard_images"
for x in os.listdir(shard_fnames):
  if "ipynb" in x:
    print(x)

.ipynb_checkpoints


In [ ]:
#/content/gdrive/My Drive/Transformer_Based_Geo-localization/resources/shard_seperated/shard_0.msg


from PIL import Image
import time
dir="/content/gdrive/My Drive/Transformer_Based_Geo-localization/resources/shard_images/"
start_time=time.time()
for x in shard_fnames[0:]: 
  try:
    imgs=x

    pil_image = PIL.Image.open(dir + imgs).convert('RGB')
    img_original = numpy.array(pil_image)
    img_data = pil_to_tensor(pil_image)
    singleton_batch = {'img_data': img_data[None].cuda()}
    output_size = img_data.shape[1:]


    #pil_image

    # Run the segmentation at the highest resolution.
    with torch.no_grad():
        scores = segmentation_module(singleton_batch, segSize=output_size)
        
    # Get the predicted scores for each pixel
    _, pred = torch.max(scores, dim=1)
    pred = pred.cpu()[0].numpy()
    

    visualize_result(img_original, pred,imgs)
  except:
    pass

end_time=time.time()-start_time
print(end_time)

0.00015997886657714844


In [ ]:
for x in shard_segment

,Class Index,Class Name,% of Image Pixels in


In [ ]:
pred

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [23, 23, 23, ..., 30, 30, 30],
       [23, 23, 23, ..., 30, 30, 30],
       [23, 23, 23, ..., 30, 30, 30]])

In [ ]:
pred_color = colorEncode(pred, colors).astype(numpy.uint8)

pred_color.flatten()

array([120, 120, 120, ...,   8, 255, 214], dtype=uint8)

In [ ]:

cnt=0
len(pred.flatten())
for x in pred.flatten():
  if x==0:
    cnt+=1

print(cnt)

89261


## Showing classes individually

To see which colors are which, here we visualize individual classes, one at a time.

In [ ]:
# Top classes in answer
predicted_classes = numpy.bincount(pred.flatten()).argsort()[::-1]
for c in predicted_classes[:5]:
    visualize_result(img_original, pred, c)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
predicted_classes = numpy.bincount(pred.flatten()).argsort()[::-1]
predicted_classes[:15]

array([ 0,  5, 23, 30, 24,  3, 14, 49, 64, 39, 15, 27, 10, 18, 66])

In [ ]:

import msgpack
import glob

from PIL import Image
from io import BytesIO

import matplotlib.pyplot as plt

def get_image(record):
        return Image.open(BytesIO(record["image"]))